<a href="https://colab.research.google.com/github/shanxS/MLPlayground/blob/main/gpt_dev_0_01_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bigram setup

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:1000])

--2023-05-09 09:20:30--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-05-09 09:20:30 (19.9 MB/s) - ‘input.txt’ saved [1115394/1115394]

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it b

In [3]:
chars = sorted(list(set(''.join(text))))
vocab_size = len(chars)

In [4]:
''.join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [5]:
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
import torch
data = torch.tensor(encode(text))

In [7]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x,y)
for i in range(block_size):
  context = x[:i+1]
  output = y[i]
  print(f'{context = } {output = } ')

tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor([47, 56, 57, 58,  1, 15, 47, 58])
context = tensor([18]) output = tensor(47) 
context = tensor([18, 47]) output = tensor(56) 
context = tensor([18, 47, 56]) output = tensor(57) 
context = tensor([18, 47, 56, 57]) output = tensor(58) 
context = tensor([18, 47, 56, 57, 58]) output = tensor(1) 
context = tensor([18, 47, 56, 57, 58,  1]) output = tensor(15) 
context = tensor([18, 47, 56, 57, 58,  1, 15]) output = tensor(47) 
context = tensor([18, 47, 56, 57, 58,  1, 15, 47]) output = tensor(58) 


In [9]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack( [data[i: i+block_size] for i in ix] )
  y = torch.stack( [data[i+1:i+block_size+1] for i in ix] )
  return x, y

xb, yb = get_batch("train")
print(xb.shape, yb.shape)
print(f'{xb=} \n {yb=}')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f'{context = }  {target = }')

torch.Size([4, 8]) torch.Size([4, 8])
xb=tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) 
 yb=tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
context = tensor([24])  target = tensor(43)
context = tensor([24, 43])  target = tensor(58)
context = tensor([24, 43, 58])  target = tensor(5)
context = tensor([24, 43, 58,  5])  target = tensor(57)
context = tensor([24, 43, 58,  5, 57])  target = tensor(1)
context = tensor([24, 43, 58,  5, 57,  1])  target = tensor(46)
context = tensor([24, 43, 58,  5, 57,  1, 46])  target = tensor(43)
context = tensor([24, 43, 58,  5, 57,  1, 46, 43])  target = tensor(39)
context = tensor([44])  target = tensor(53)
context = tensor([44, 53])  target = tensor(56)
context = tensor([44, 53, 56])  target = tensor(1)
context = t

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLangugageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    '''
    1st argument is size of dict
    2nd argument is size of embedding for each input token
    So the dim is vocab_size x C, where C is Channel
    '''
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, input, target = None):
    '''
    This is the output. Literally just dot product of input (B x T) and embedding table (vocab_size x channel) to get embedding
    I think it internally converts input into one hot encoding of size (B x T x vocab_size)
    B: number of inputs
    T: size of each input
    C: channel
    '''
    # print(input)
    # print(target)
    
    logits = self.token_embedding_table(input) # (B x T x C)

    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T)
      loss = F.cross_entropy(logits, target)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)
      logits = logits[:, -1, :] # I dont understand why last index is predicted token
      logits = logits.exp()
      prob = logits / logits.sum(1, keepdims = True)
      idx_next = torch.multinomial(prob, num_samples=1, replacement=True)
      idx = torch.cat((idx, idx_next), dim =1)
    return idx


m = BigramLangugageModel(vocab_size)
# logits, loss = m(xb, yb)

In [11]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [12]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [13]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  loss.backward()
  optimizer.step()

print(loss)

tensor(2.7662, grad_fn=<NllLossBackward0>)


In [14]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long), max_new_tokens = 400)[0].tolist()))


I oteng h hasbe pave pirance
RDe hicomyonthar's
PES:
ARod ith heno hatucenonthioneir thondy, y heltieiengerofuresssit ey
KIN d pe wither vouprrouthers nowl t,
K:

My hind thid pig t ouchos tes; st yo hind wotin grotonear 'so it t jod weancothanan hay. t le n frids, r loncave w hollular s O:
HIs; ht anje caike ineent.

Lavinde.
athave l.
KEONGBUThandspo be y,-hedarwnod y scar t aridesar, fre'shenou


# Self attention

In [16]:
torch.manual_seed(1337)
B,T,C = 8,4,2
x = torch.randn(B,T,C) # logits after we send input into embedding layer

### approach 1: manually average past tokens

In [54]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1] # t+1 to include current token
    xbow[b, t] = torch.mean(xprev, 0) # mean along rows, keeping col const

print(x[0])
print(xbow[0])

tensor([[ 0.1588, -2.3106],
        [ 0.3227,  1.5431],
        [-1.3924,  1.3114],
        [ 1.3805,  0.3544]])
tensor([[ 0.1588, -2.3106],
        [ 0.2407, -0.3837],
        [-0.3037,  0.1813],
        [ 0.1174,  0.2246]])


### approach 2: matrix multiplication

In [55]:
# to being with, lets keep math simple we'll stick to 2D
tmp = torch.tril(torch.ones(3,3)) # this helps with summing across rows, keeping col const
tmp = tmp / tmp.sum(dim=1, keepdims=True) # this helps with average
xtmp = torch.randint(0,10,(3,2)).float()
print(f'{xtmp = } \n {tmp = } \n {tmp@xtmp = }')

xtmp = tensor([[6., 2.],
        [6., 7.],
        [7., 0.]]) 
 tmp = tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) 
 tmp@xtmp = tensor([[6.0000, 2.0000],
        [6.0000, 4.5000],
        [6.3333, 3.0000]])


In [56]:
# final version
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(dim = 1, keepdims = True)
print(f'{wei.shape=}')

xbow2 = wei @ x # note that wei does not have Batch dim but this works anyway
print(f'{x[0]=} \n {xbow2[0]=}')
torch.allclose(xbow, xbow2)

wei.shape=torch.Size([4, 4])
x[0]=tensor([[ 0.1588, -2.3106],
        [ 0.3227,  1.5431],
        [-1.3924,  1.3114],
        [ 1.3805,  0.3544]]) 
 xbow2[0]=tensor([[ 0.1588, -2.3106],
        [ 0.2407, -0.3837],
        [-0.3037,  0.1813],
        [ 0.1174,  0.2246]])


True

### approach 3: using softmax

In [59]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T) # matrix that will eventually decide which part of history impacts a given token most
wei = wei.masked_fill(tril == 0, float('-inf')) # enforces that nothing is read from future
wei = torch.softmax(wei, dim=-1) # averages all cols of a row
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True